# Modèle d'Heston

Ce notebook présente plusieurs portions du code que nous avons utilisé pour calibrer le modèle d'Heston. On présentera l'obtention des données, et les algorithmes utilisés pour la calibration du modèle.

In [1]:
#### Importation des librairies
library(MASS)
library(httr)
library(jsonlite)
library(rgl)

## Récupération des données

On présente ici le code afin d'obtenir les données sur les options. On peut modifier le sous-jacent étudié et le cours actuel du sous-jacent est à rentrer manuellement.

In [ ]:
#### données (sous-jacent,date, etc..)

actif_sous_jacent <- "AAPL"

date_initiale <- as.numeric (as.POSIXct (Sys.time ()))

S_ <- 161.79


#On construit la base de données (automatisation)

url <- "https://yh-finance.p.rapidapi.com/stock/v2/get-options"

queryString <- list(
  symbol = actif_sous_jacent,
  region = "US"
)

response <- VERB("GET", url, add_headers('X-RapidAPI-Host' = 'yh-finance.p.rapidapi.com', 'X-RapidAPI-Key' = ''), query = queryString)

#il faut insérer X-RapidAPI-Key (clé personelle)

data<- fromJSON(rawToChar(response$content))

echeances <- data$meta$expirationDates #accéder aux dates

######
transi <- data$contracts$calls[,c("strike","ask","bid","expiration")]
options <- cbind(transi$strike$raw,transi$ask$raw,
                 transi$bid$raw,transi$expiration$raw)
for(k in 1:length(echeances)){
  queryString <- list(
    symbol = actif_sous_jacent,
    date = echeances[k],
    region = "US"
  )
  response <- VERB("GET", url, add_headers('X-RapidAPI-Host' = 'yh-finance.p.rapidapi.com', 'X-RapidAPI-Key' = '8bbb8947bdmsh782b2f6aa73cfc4p12800ajsn414e973da860'), query = queryString)
  databis<- fromJSON(rawToChar(response$content))
  
  transi <- databis$contracts$calls[,c("strike","ask","bid","expiration")]
  optionsbis <- cbind(transi$strike$raw,transi$ask$raw,
                      transi$bid$raw,transi$expiration$raw)
  
  
  options <- rbind(options,optionsbis)
}

#cette base de données peut contenir des N.A

options<-options[!is.na(options[,2]),]



### Illustration  de la nappe de prix observée

In [ ]:
#### Essai nappe prix

x<-options[,1]
y<-(options[,4] - rep(date_initiale,length(x)))/31536000
z_mid_price<-(options[,2]+options[,3])/2
plot3d(x,y,z_mid_price,type="p",col="purple",xlab="strike",ylab="echeance",zlab="prix" )

## Implémentation de la formule du prix d'un call

In [ ]:
##### Fonctions donnant la formule de prix

i=complex(1,real=0,imaginary=1)

#Dans la suite du code:
#       volsj --> volatilité du sous-jacent (dans BS)
#       rho --> coeff de corrélation entre les deux browniens
#       sigma --> volatilité de la volatilité du sous-jacent
#       lambda --> prix de marché du risque de volatilité
#       tau --> durée avant l'écéhance (tf - t)
#       k --> force de rappel de la variance du sous-jacent
#       teta --> variance asymptotique du sous-jacent
#       V --> variance du sous-jacent à l'instant 0 




#On définit les fonctions pour pricer dans le modèle d'Heston

#Fonctions qui interviennent dans le prix du call
#dans le modèle d'Heston



d <- function(rho,sigma,b,u,z){
  return((   ((rho*sigma*z*i - b)^2) - (sigma^2)*(2*u*z*i - (z^2)) )^(1/2))
}

g <- function(rho,sigma,b,u,z){
  d_=d(rho,sigma,b,u,z)
  return((b - rho*sigma*z*i + d_ )/(b - rho*sigma*z*i - d_))
}

B <- function(rho,sigma,b,u,tau,z){
  d_=d(rho,sigma,b,u,z)
  g_=g(rho,sigma,b,u,z)
  T1=((b-rho*sigma*z*i+ d_)/(sigma^2))
  T2=(1-exp(d_*tau))/(1-g_*exp(d_*tau))
  return (T1*T2)
}

A<- function(rho,sigma,b,u,r,tau,k,teta,z){
  a=k*teta
  d_=d(rho,sigma,b,u,z)
  g_=g(rho,sigma,b,u,z)
  T2=(1 - g_*exp(d_*tau))/(1 - g_)
  return(r*z*i*tau + (a/(sigma^2))*((b-rho*sigma*z*i + d_)*tau - 2*log(T2) ) )
}

f <- function(S,K,V,rho,sigma,b,u,r,tau,k,teta,z){
  A_=A(rho,sigma,b,u,r,tau,k,teta,z)
  B_= B(rho,sigma,b,u,tau,z)
  x=log(S)
  return(exp(A_ + B_*V +i*z*x))
}

f_c_i <- function(z,S,K,V,rho,sigma,b,u,r,tau,k,teta){
  f_=f(S,K,V,rho,sigma,b,u,r,tau,k,teta,z)
  return(Re(((exp(-i*z*log(K)))*f_)/(i*z)))
}

prix_heston <- function(S,K,V,rho,sigma,r,tf,t,k,teta,lambda){
  tau=(tf-t)/31536000
  u1=1/2
  u2=-1/2
  b1=k- rho*sigma + lambda
  b2=k + lambda
  
  pas<-1/100
  I1<-0
  I2<-0
  X<-seq(0,100,pas)
  for(j in 2:10001){
    Z<-X[j]
    I1<-I1 + pas*f_c_i(Z,S,K,V,rho,sigma,b1,u1,r,tau,k,teta)
    I2<-I2 + pas*f_c_i(Z,S,K,V,rho,sigma,b2,u2,r,tau,k,teta)
  }
  
  P1= (1/2) + (1/pi)*I1
  P2= (1/2) + (1/pi)*I2
  
  return (S*P1 - K*exp(-r*tau)*P2)
}


## Calibration du modèle

Ici, on implémente à proprement parler la distance entre les prix observés et les prix prédits par le modèle ainsi qu'un algorithme servant à minimiser cette distance.

In [ ]:
#### Problème de minimisation




omega <- function(K){
  return(exp(-10*abs((S_/K)-1)))
}



r_opti<-0.02469  # ln(1+R) avec R le taux sans risque américain d'échéance 2 ans (nos options ont généralement une maturité inférieure à 2 ans)


erreur_quadratique_heston <- function(rho,k,teta,V,sigma,lambda){
  return(sum(omega(options[,1])*((((options[,2]+options[,3])/2)-prix_heston(S_,options[,1],V,rho,sigma,r_opti,
                                                                            options[,4],date_initiale,k,teta,lambda))^2)))
}
#Cette fonction renvoie la somme des différences au carré
#des écarts entre le prix observé et le prix prédit par Heston en fonction
#des différents paramètres.

G <- function(p){
  return(erreur_quadratique_heston(p[1],p[2],p[3],p[4],p[5],p[6]))
  
} 
#Cette fonction renvoie la même chose que
#erreur_quadratique_heston mais prend en entrée
#un vecteur.


gradient_heston <- function(p){
  M<-rep(0,6)
  R<-G(p)
  M[1]<-100*(erreur_quadratique_heston(p[1]+1/100,p[2],p[3],p[4],p[5],p[6])-R)
  M[2]<-100*(erreur_quadratique_heston(p[1],p[2]+1/100,p[3],p[4],p[5],p[6])-R)
  M[3]<-100*(erreur_quadratique_heston(p[1],p[2],p[3]+1/100,p[4],p[5],p[6])-R)
  M[4]<-100*(erreur_quadratique_heston(p[1],p[2],p[3],p[4]+1/100,p[5],p[6])-R)
  M[5]<-100*(erreur_quadratique_heston(p[1],p[2],p[3],p[4],p[5]+1/100,p[6])-R)
  M[6]<-100*(erreur_quadratique_heston(p[1],p[2],p[3],p[4],p[5],p[6]+1/100)-R)
  return(M)
}

#3min30

projecteur_K <- function(p){
  pk<-rep(0,length(p))
  pk[1]<-max(-1,min(p[1],1))
  pk[2]<-max(0,p[2])
  pk[3]<-max(0,p[3])
  pk[4]<-max(0,p[4])
  pk[5]<-max(0,p[5])
  pk[6]<-max(0,p[6])
  return(pk)
}

descente_gradient_pas_fixe_heston <- function(départ,iterations){
    j=0
    epsilon<-1/10
    p0<-départ
    gr<-gradient_heston(p0)
    if(gr==rep(0,6)){return(p0)}
    pas<-1/(100*max(abs(gr)))
    p1<-projecteur_K(p0 - pas*gr)
  while((abs(G(p0)-G(p1))>epsilon)&&(j<iterations)){
    p0<-p1
    gr<-gradient_heston(p0)
    if(gr==rep(0,6)){return(p0)}
    pas<-1/(100*max(abs(gr)))
    p1<-projecteur_K(p0 - pas*gr)
    j=j+1
  }
  print(j)
  return(p0)
}

#p_opti<-descente_gradient_pas_fixe_heston(c(-0.5,1,1,1,1,1),200)[1:6]

p_opti<-c(-0.3537,0.1761,2.7343,0,0.9107,3.339) #vecteur (rho,k,teta,V,sigma,lambda)


## Indicateurs de distance  

On définit ici le RWMSE (Root Weight Mean Square Error) qui mesure l'écart entre les prédictions et la réalité

In [ ]:
##### Calcul du RWMSE

poids <- function(K){
  return(omega(K)/sum(omega(options[,1])))
}

RWMSE <- function(rho,k,teta,V,sigma,lambda){
  return(sqrt(sum(poids(options[,1])*((((options[,2]+options[,3])/2)-prix_heston(S_,options[,1],V,rho,sigma,r_opti,
                                                                     options[,4],date_initiale,k,teta,lambda))^2))))
}

### Illustration de la nappe de prix prédite et de la nappe de prix observée

In [ ]:
##### Nappes de prix

prix<- prix_heston(S_,options[,1],0,-0.3537,0.9107,0.03,options[,4],date_initiale,0.1761,2.7344,3.339)
z<-(options[,2]+options[,3])/2

open3d()
plot3d(x,y,z,type="p",col="purple",xlab="strike",ylab="echeance",zlab="prix" )
open3d()
plot3d(x,y,prix,type="p",col="red",xlab="strike",ylab="echeance",zlab="prix")


## Générateur de trajectoires sous la MME

Nous définissons une fonction qui génère des trajectoires du sous-jacent à partir des paramètres du modèle calibré.

In [ ]:
#################### trajectoires du sous jacent



graph1<-function(S_0){
  X<-seq(0,1,1/365)
  Y1<-rep(0,length(X))
  Y2<-rep(0,length(X))
  Y1[1]<-S_0
  Y2[1]<- p_opti[4]
  for(k in 2:length(Y1)){
    U<-mvrnorm(n=1,mu=c(0,0),Sigma=rbind(c(1/365,p_opti[1]/365),c(p_opti[1]/365,1/365)))
    Y1[k]<-Y1[k-1]+ r_opti*Y1[k-1]*(X[k]-X[k-1]) + sqrt(Y2[k-1])*Y1[k-1]*U[1]
    Y2[k]<-max(Y2[k-1] + (p_opti[2]+p_opti[6])*(((p_opti[2]*p_opti[3])/(p_opti[2]+p_opti[6]))-Y2[k-1])*(X[k]-X[k-1]) + p_opti[5]*sqrt(Y2[k-1])*U[2],0) 
    
  }
  plot(X,Y1,type="l")
  
}

#(-0.3537,0.1761,2.7343,0,0.9107,3.339) vecteur (rho,k,teta,V,sigma,lambda) optimal

#en univers risque neutre : k* = k + lambda ; teta*=kteta/k*


